In [48]:
#!/usr/bin/python



In [49]:
import json
import re
import os
import sys

In [50]:
def parse_line(line):
    """Split the line into a tuple
    (depth, feature, comparator, value, classification/None,numeric,rule)"""
    #print(line)
    split = [ l for l in line.split() if len(l) > 0 ]
    depth = 0
    rule = 0
    condition = ""
    punc = ":"
    feature = ""
    comparator =[">", "<", "<=", ">=","="]
    #for part in split:
       # if part == "|":
         #   depth += 1 
        # else:
           # break
    for part in split:
        #print(part)
        if part == "|":
            condition = ""
            depth = depth +1
            
        elif part in comparator:
            feature = condition
            if part == "=":
                part = "=="
            else:    
                condition = condition + " " + part + " "
            
        elif punc in part :
            part = part.replace(punc," ")
            condition = condition + part
            #print(condition)
            break
        else :
            condition = condition + part
            #print(condition)
            
    #if len(split) > depth + 4: 
    if ("SKIP" in split):
        rule = 1
    elif ("Low" in split):
        rule = 1
    elif ("HIGH" in split):
        rule = 1
    elif ("MEDIUM" in split):
        rule = 1
    else:
        rule = 0
    return (depth,feature,rule,condition)
    #return (depth,rule)

In [51]:
def parse_tree(lines):
    i=0
    co=0
    parents = [0 for l in range(20)]
    conditions =[] 
    rules =[]
    ruleCount =0
    features = []
    #ffeature = "feature.txt"
    #ff = open(ffeature, "w")
   # print(lines)
    while i < len(lines):
            line = lines[i]
            
            print (line)
            depth,feature,rule,condition = parse_line(line)
            #depth,rule = parse_line(line)
            conditions.append(condition)
            features.append(feature)
            index = int(depth)
            print (rule)
            #print ("index = " + str(index) + str(i))
            GRule = "( "
            parents[index] = i
            k=0
            if rule == 1 :
                for k in range(index):
                    
                    GRule = GRule + conditions[parents[k]]+ " && "
                    
                GRule = GRule + conditions[parents[k +1]] + " )"
                print (GRule)
                rules.append(GRule)
            i = i + 1                 
            
                
            
           
            
    return(rules)

In [56]:
#input_filename = "model_exp1.txt"
#input_filename = "sample.txt"
input_filename = "Stage-1_DT.txt"
if os.path.isfile(input_filename):
            f = open(input_filename)
            lines = f.readlines()
            # print (lines)
            f.close()
else:
            raise Exception("Error : File %s not found!" % input_filename)
    #else:
       # lines = sys.stdin.readlines()

if not lines:
        raise Exception("Error : Empty input!")
#tree_lines = get_tree_lines(lines)
#tree = parse_tree(tree_lines)
rules = parse_tree(lines)
#print (rules)
#features = get_feature(lines)
empdata = ""
if os.path.isfile("employeedata_V16.txt"):
            ff = open("employeedata_V16.txt")
            empData = ff.readlines()
            #print (empData)
            for i in range(len( empData)):
                
                empData[i]= (empData[i].replace(" ","")).strip()
                
                #print (empData[i])
            
            
            

ff.close()
#write the program file BussinessRule.java
output_filename = "BusinessRuleJuly22_2_1"
o_filename = output_filename + ".java" 
fout = open(o_filename, "w")
#fout.write ("#!/usr/bin/python  \n import os \n import sys \n")
fout.write("package weka.train; \n \n")
fout.write("import java.io.FileReader; \n" )        
fout.write("import java.io.IOException;\n" )
fout.write("import java.io.BufferedReader; \n" )
fout.write("import java.io.BufferedWriter; \n")
fout.write("import java.io.File;\n")
fout.write("import java.io.FileWriter;\n")
fout.write("import java.io.FileNotFoundException;\n")
fout.write("import weka.core.Instances;\n")
fout.write("import weka.core.Instance;\n \n \n")           
fout.write ("public class " + output_filename + " {\n\n")






#rulecheker starts

    
fout.write ("    public static void ruleChecker (String line, FileWriter fileWriter)\n"
  + "    throws Exception {\n\n" )


fout.write("      String[] emp = line.split(,); \n")
#fout.write("      System.out.println(emp[0]); \n")
         
for i in range(len( empData)):
    #fout.write("double " + empData[i] + " = inst.value(" + str(i) + ")" )
    fout.write("String " + empData[i] + " = emp[" + str(i) + "]" )          
               
    fout.write(" ; \n")
  
           
fout.write("\n \n") 
c = 0
for k in rules:
    if (c == 0) :
        fout.write(" if ")
    
    else:
        fout.write ("else if")
    c = c + 1    
    fout.write(k)
    fout.write ("\n { \n   ")
    #fout.write ("\n")
    #fout.write ("\t")
    fout.write (" System.out.println \" " + k +" \" ; \n" )
    #file write        
    # writing out to csv file :  emp[len(emp)-1] supposed to be the actual class     
    fout.write ("fileWriter.append(emp[0] + \",\" + emp[len(emp)-1] + \",\"  + \" " + k + " \"); \n ")       
    fout.write ("\n } \n")

fout.write ("}// ruleChecker \n")
#rulechecker ends            
# main method starts           
fout.write("public static void main(String args[])  throws Exception {\n\n" ) 
            
# input file declaration
fout.write("datafile = \"India_Annotated_Data_v16_predictions_historical.csv\" ; \n ")
fout.write ("BufferedReader br = new BufferedReader(new FileReader(datafile)); \n")
            
#Output file declaration
fout.write("reasonfileName = \"reasonReco.csv\"; \n")
fout.write("FileWriter fileWriter = null; \n \n ")
      
#fout.write("       try { \n"
fout.write("           fileWriter = new FileWriter(reasonfileName); \n ")
fout.write("           //Write the CSV file header \n ")
fout.write("           fileWriter.append(A,B,C,D); \n ")
# file read
fout.write("String line = \" \" ; \n ")
fout.write("while((line = br.readLine()) != null){ \n ")
fout.write ("    rulechecker(line,fileWriter); \n")
fout.write ("}  // while  \n")
fout.write("}\n // main") 
# main method ends
fout.write("} // class \n")
fout.close()


Years in Band <= 0.9

0
|   Salary Increase Amount 02 <= 5650.08

0
|   |   Salary Increase % 02 <= 6.94

0
|   |   |   Years in Service <= 1

0
|   |   |   |   AR Local Curr <= 372750

0
|   |   |   |   |   HR Band = 7A: SKIP (0.0)

1
( YearsinBand <= 0.9 && SalaryIncreaseAmount02 <= 5650.08 && SalaryIncrease%02 <= 6.94 && YearsinService <= 1 && ARLocalCurr <= 372750 && HRBand = 7A  )
|   |   |   |   |   HR Band = 6A

0
|   |   |   |   |   |   AR Local Curr <= 323463: SKIP (8.7)

1
( YearsinBand <= 0.9 && SalaryIncreaseAmount02 <= 5650.08 && SalaryIncrease%02 <= 6.94 && YearsinService <= 1 && ARLocalCurr <= 372750 && HRBand = 6A && ARLocalCurr <= 323463  )
|   |   |   |   |   |   AR Local Curr > 323463: MEDIUM (7.54/3.48)

1
( YearsinBand <= 0.9 && SalaryIncreaseAmount02 <= 5650.08 && SalaryIncrease%02 <= 6.94 && YearsinService <= 1 && ARLocalCurr <= 372750 && HRBand = 6A && ARLocalCurr > 323463  )
|   |   |   |   |   HR Band = 7B: SKIP (0.0)

1
( YearsinBand <= 0.9 && SalaryIncreaseA

In [36]:
def get_feature(lines):
    i=0
    features = [] 
    while i < len(lines):
            line = lines[i]
            depth, feature, comparator, value, classif,numeric,rule,condition = parse_line(line)
            #print (feature )
            if feature not in features:
                features.append(feature)
                
    return (features)